In [17]:
from FR3Py.robot.model import PinocchioModel
from liegroups import SO3
import numpy as np

In [ ]:
from FR3Py.robot.interface import FR3Real

In [ ]:
robot = FR3Real(robot_id='fr3')

In [ ]:
model = PinocchioModel()

In [ ]:
robot.getStates()

In [ ]:
import time
state = robot.getStates()
info = model.getInfo(state['q'], state['dq'])
R0 = info['R_EE']
Kv = 3*np.eye(3)
Kp = 20*np.eye(3)
while True:
    time.sleep(0.002)
    state = robot.getStates()
    info = model.getInfo(state['q'], state['dq'])
    J = info['J_EE'][:, :7]
    Jr = J[3:, :] 
    omega = Jr@state['dq'][:7]
    rot_error = SO3(R0@info['R_EE'].T).log()
    tau = Jr.T@(Kp@rot_error - Kv*omega) 
    robot.sendCommands(np.hstack([tau, np.zeros(2)]))